Solve linear systems using straightforward methods (Book 1, p. 94, ex. 11.5).

In [1]:
import numpy as np
from numpy.linalg import solve

# Common parts

In [2]:
def residual(A, b, x):
    """Calculating solution error.

    Returns:
        res_ver (ndarray): b - Ax
    """
    res_vec = b - np.dot(A, x)
    return res_vec

In [3]:
# Initial test matrix.
A = np.array([[7.35272,  0.88255,  -2.270052],
              [0.88255,  5.58351,  0.528167],
              [-2.27005, 0.528167, 4.430329]])

# Test matrix where c[0][0] = a[0][0]*10^(-8)
C = np.array([[7.35272e-8,  0.88255,  -2.270052],
              [0.88255,     5.58351,  0.528167],
              [-2.27005,    0.528167, 4.430329]])

b = np.array([[1.],
              [0.],
              [0.]])

# Gaussian elimination

In [4]:
def gaussian_elimination(A, b, eps=1e-5):
    """Solving linear system using gaussian elimination.

    Args:
        A (ndarray<ndarray, ndarray>): matrix of coefficents.
        b (ndarray): vector of values.
        eps (float): all values below eps equivalent to zero.

    Returns:
        x (ndarray): solution.
    """

    # Getting matrix shape.
    n = A.shape[0]

    # Merging coefficents with values.
    Ab = np.concatenate((A, b), axis=1)
    print("\nMerged matrix: \n{}".format(Ab))

    # Making upper triangular matrix.
    for k in range(0, n):

        # Dividing all row alements after
        # diagonal element on diagonal
        # element.
        tmp = Ab[k][k]
        if np.abs(tmp) < eps:
            print("\nElement Ab[{}][{}]={} smaller than eps={}.".format(
                k, k, tmp, eps))
        for j in range(k, n + 1):
            Ab[k][j] = Ab[k][j] / tmp

        # Substracting top element multiplied
        # by 1st element in row from each
        # element.
        for i in range(k + 1, n):
            tmp = Ab[i][k]
            for j in range(k, n + 1):
                Ab[i][j] = Ab[i][j] - Ab[k][j] * tmp

    print("\nUpper triangular matrix: \n{}".format(Ab))

    # Solve equation for an upper
    # triangular matrix Ab.
    x = np.zeros((3, 1))
    for i in range(n - 1, -1, -1):
        x[i] = Ab[i][n] / Ab[i][i]
        for k in range(i - 1, -1, -1):
            Ab[k][n] -= Ab[k][i] * x[i]

    print("\nSolution: \n{}".format(x))

    # Calculating error.
    res_vec = residual(A, b, x)
    print("\nResidual vector: \n{}".format(res_vec))

    return x

### test 1

In [5]:
_ = gaussian_elimination(A, b)


Merged matrix: 
[[ 7.35272   0.88255  -2.270052  1.      ]
 [ 0.88255   5.58351   0.528167  0.      ]
 [-2.27005   0.528167  4.430329  0.      ]]

Upper triangular matrix: 
[[ 1.          0.12003041 -0.30873636  0.13600409]
 [ 0.          1.          0.14616723 -0.02191305]
 [ 0.          0.          1.          0.09032103]]

Solution: 
[[ 0.16810435]
 [-0.03511502]
 [ 0.09032103]]

Residual vector: 
[[  0.00000000e+00]
 [ -2.08166817e-17]
 [ -5.55111512e-17]]


### test 2

In [6]:
_ = gaussian_elimination(C, b)


Merged matrix: 
[[  7.35272000e-08   8.82550000e-01  -2.27005200e+00   1.00000000e+00]
 [  8.82550000e-01   5.58351000e+00   5.28167000e-01   0.00000000e+00]
 [ -2.27005000e+00   5.28167000e-01   4.43032900e+00   0.00000000e+00]]

Element Ab[0][0]=7.35272e-08 smaller than eps=1e-05.

Upper triangular matrix: 
[[  1.00000000e+00   1.20030411e+07  -3.08736359e+07   1.36004091e+07]
 [  0.00000000e+00   1.00000000e+00  -2.57215256e+00   1.13308088e+00]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00  -3.82677011e-01]]

Solution: 
[[-0.71223358]
 [ 0.14877722]
 [-0.38267701]]

Residual vector: 
[[  0.00000000e+00]
 [  1.65219294e-09]
 [  2.11953721e-09]]


# Gaussian elimination with main element choose

In [7]:
def gaussian_elimination_choose(A, b, type="col", eps=1e-5):
    """Solving linear system using gaussian elimination with main element choosing.

    Args:
        A (ndarray<ndarray, ndarray>): matrix of coefficents.
        b (ndarray): vector of values.
        type (str): one of three: row  (choose max element within a row)
                                  col  (choose max element within a column)
                                  both (choose max element within both row and column)
        eps (float): all values below eps equivalent to zero.

    Returns:
        x (ndarray): solution.
    """

    # Getting matrix shape.
    n = A.shape[0]

    # Merging coefficents with values.
    Ab = np.concatenate((A, b), axis=1)
    print("\nMerged matrix: \n{}".format(Ab))

    # Making upper triangular matrix.
    for k in range(0, n):

        # Search for maximum in this column.
        max_elem = abs(Ab[k][k])
        max_row = k
        for i in range(k + 1, n):
            if abs(Ab[i][k]) > max_elem:
                max_elem = abs(Ab[i][k])
                max_row = i

        # Swap maximum row with current row (column by column).
        for i in range(0, n + 1):
            tmp = Ab[max_row][i]
            Ab[max_row][i] = Ab[k][i]
            Ab[k][i] = tmp
            
        # Dividing all row alements after
        # diagonal element on diagonal
        # element.
        tmp = Ab[k][k]
        if np.abs(tmp) < eps:
            print("\nElement Ab[{}][{}]={} smaller than eps={}.".format(
                k, k, tmp, eps))
        for j in range(k, n + 1):
            Ab[k][j] = Ab[k][j] / tmp

        # Substracting top element multiplied
        # by 1st element in row from each
        # element.
        for i in range(k + 1, n):
            tmp = Ab[i][k]
            for j in range(k, n + 1):
                Ab[i][j] = Ab[i][j] - Ab[k][j] * tmp

    print("\nUpper triangular matrix: \n{}".format(Ab))

    # Solve equation for an upper
    # triangular matrix Ab.
    x = np.zeros((3, 1))
    for i in range(n - 1, -1, -1):
        x[i] = Ab[i][n] / Ab[i][i]
        for k in range(i - 1, -1, -1):
            Ab[k][n] -= Ab[k][i] * x[i]

    print("\nSolution: \n{}".format(x))

    # Calculating error.
    res_vec = residual(A, b, x)
    print("\nResidual vector: \n{}".format(res_vec))

    return x

### test 1

In [8]:
_ = gaussian_elimination_choose(A, b)


Merged matrix: 
[[ 7.35272   0.88255  -2.270052  1.      ]
 [ 0.88255   5.58351   0.528167  0.      ]
 [-2.27005   0.528167  4.430329  0.      ]]

Upper triangular matrix: 
[[ 1.          0.12003041 -0.30873636  0.13600409]
 [ 0.          1.          0.14616723 -0.02191305]
 [ 0.          0.          1.          0.09032103]]

Solution: 
[[ 0.16810435]
 [-0.03511502]
 [ 0.09032103]]

Residual vector: 
[[  0.00000000e+00]
 [ -2.08166817e-17]
 [ -5.55111512e-17]]


### test 2

In [9]:
_ = gaussian_elimination_choose(C, b)


Merged matrix: 
[[  7.35272000e-08   8.82550000e-01  -2.27005200e+00   1.00000000e+00]
 [  8.82550000e-01   5.58351000e+00   5.28167000e-01   0.00000000e+00]
 [ -2.27005000e+00   5.28167000e-01   4.43032900e+00   0.00000000e+00]]

Upper triangular matrix: 
[[ 1.         -0.23266756 -1.9516438  -0.        ]
 [ 0.          1.          0.38878014  0.        ]
 [ 0.          0.          1.         -0.38267701]]

Solution: 
[[-0.71223358]
 [ 0.14877722]
 [-0.38267701]]

Residual vector: 
[[  0.00000000e+00]
 [ -1.11022302e-16]
 [  0.00000000e+00]]


# LU decomposition

In [10]:
def LU(A):
    """Making an LU matrix decomposition.

    Args:
        A (ndarray<ndarray, ndarray>): matrix to decompose.

    Returns:
        L, U (ndarray<ndarray, ndarray>): lower and upper triangular matrixes.
    """

    # Getting matrix shape.
    n = A.shape[0]
    
    # Generating empty matrixes.
    L = np.zeros((n, n))
    U = np.zeros((n, n))

    # Counting LU coefficents.
    for i in range(0, n):

        for j in range(0, n):
            sum = 0
            for k in range(0, i):
                sum += L[j][k] * U[k][i]
            L[j][i] = A[j][i] - sum
            
        for j in range(0, n):
            sum = 0
            for k in range(0, i):
                sum += L[i][k] * U[k][j]
            U[i][j] = (A[i][j] - sum) / L[i][i]

    return L, U

In [11]:
def LU_system_solution(A, b):
    """Solving linear system using LU decomposition.

    Args:
        A (ndarray<ndarray, ndarray>): matrix of coefficents.
        b (ndarray): vector of values.

    Returns:
        x (ndarray): solution.
    """
    
    print("\nGiven matrix A: \n{}".format(A))
    print("\nb: \n{}".format(b))
    
    L, U = LU(A)
    print("\nL: \n{}".format(L))
    print("\nU: \n{}".format(U))
    
    y = solve(L, b)
    print("\ny: \n{}".format(y))
    x = solve(U, y)
    print("\nSolution: \n{}".format(x))

    # Calculating error.
    res_vec = residual(A, b, x)
    print("\nResidual vector: \n{}".format(res_vec))
    
    return x

### test 1

In [12]:
_ = LU_system_solution(A, b)


Given matrix A: 
[[ 7.35272   0.88255  -2.270052]
 [ 0.88255   5.58351   0.528167]
 [-2.27005   0.528167  4.430329]]

b: 
[[ 1.]
 [ 0.]
 [ 0.]]

L: 
[[ 7.35272     0.          0.        ]
 [ 0.88255     5.47757716  0.        ]
 [-2.27005     0.80064203  3.6124544 ]]

U: 
[[ 1.          0.12003041 -0.30873636]
 [ 0.          1.          0.14616723]
 [ 0.          0.          1.        ]]

y: 
[[ 0.13600409]
 [-0.02191305]
 [ 0.09032103]]

Solution: 
[[ 0.16810435]
 [-0.03511502]
 [ 0.09032103]]

Residual vector: 
[[  0.00000000e+00]
 [ -2.08166817e-17]
 [ -5.55111512e-17]]


### test 2

In [13]:
_ = LU_system_solution(C, b)


Given matrix A: 
[[  7.35272000e-08   8.82550000e-01  -2.27005200e+00]
 [  8.82550000e-01   5.58351000e+00   5.28167000e-01]
 [ -2.27005000e+00   5.28167000e-01   4.43032900e+00]]

b: 
[[ 1.]
 [ 0.]
 [ 0.]]

L: 
[[  7.35272000e-08   0.00000000e+00   0.00000000e+00]
 [  8.82550000e-01  -1.05932783e+07   1.82223325e-09]
 [ -2.27005000e+00   2.72475039e+07   4.40876544e+01]]

U: 
[[  1.00000000e+00   1.20030411e+07  -3.08736359e+07]
 [ -0.00000000e+00   1.00000000e+00  -2.57215256e+00]
 [  0.00000000e+00   4.13320527e-11   1.00000000e+00]]

y: 
[[  1.36004091e+07]
 [  1.13308088e+00]
 [ -3.82677011e-01]]

Solution: 
[[-0.71223358]
 [ 0.14877722]
 [-0.38267701]]

Residual vector: 
[[ -2.22044605e-16]
 [  1.31102904e-09]
 [ -3.75871023e-09]]


# Inverse matrix

In [14]:
def inverse_matrix(A):
    """Inverting a matrix.

    Args:
        A (ndarray<ndarray, ndarray>): matrix to invert.

    Returns:
        X (ndarray<ndarray, ndarray>): inverse matrix.
    """
    
    # Getting matrix shape.
    n = A.shape[0]
    print("\nGiven matrix A: \n{}".format(A))
    
    L, U = LU(A)
    print("\nL: \n{}".format(L))
    print("\nU: \n{}".format(U))
    
    I = np.identity(n)
    
    X = solve(np.dot(L, U), I)
    print("\nSolution: \n{}".format(X))
    
    check = np.dot(A, X)
    print("\nA * A^(-1): \n{}".format(check))
    
    return X

In [15]:
_ = inverse_matrix(A)


Given matrix A: 
[[ 7.35272   0.88255  -2.270052]
 [ 0.88255   5.58351   0.528167]
 [-2.27005   0.528167  4.430329]]

L: 
[[ 7.35272     0.          0.        ]
 [ 0.88255     5.47757716  0.        ]
 [-2.27005     0.80064203  3.6124544 ]]

U: 
[[ 1.          0.12003041 -0.30873636]
 [ 0.          1.          0.14616723]
 [ 0.          0.          1.        ]]

Solution: 
[[ 0.16810435 -0.03511503  0.0903211 ]
 [-0.03511502  0.18847669 -0.04046203]
 [ 0.09032103 -0.04046202  0.2768201 ]]

A * A^(-1): 
[[  1.00000000e+00   1.90535357e-17  -4.00690908e-17]
 [ -9.53110656e-18   1.00000000e+00  -2.80042633e-17]
 [ -5.39921752e-18  -4.12010219e-17   1.00000000e+00]]
